### Teilaufgabe 1
Recherchieren und erklären Sie die Begriffe "True Positive Rate" (TPR) und "False Positive Rate" (FPR).

Können Sie einen Zusammenhang herstellen zwischen diesem Begriffspaar und dem Begriffspaar Sensitivität/Spezifität?

### Teilaufgabe 2
Arbeiten Sie das Tutorial ROC & AUC (mlu-explain.github.io) durch und erläutern Sie, wie Sie die Achsenbeschriftung im ROC-Graphen durch Begriffe ersetzen können, die Sie aus der Vorlesung kennen.

### Teilaufgabe 3
Betrachten Sie den Train/Validation/Test-Split im Heart-Datensatz aus Aufgabe 5 sowie den dort trainierten DecisionTreeClassifier mit bestmöglichen Hyperparametern.

Zeichnen Sie die ROC-Kurve und berechnen Sie den AUC-Wert.

Hinweis: Sie können ROC-Kurve und AUC selber berechnen oder hierzu eine geeignete sklearn-Funktionalität verwenden.